In [2]:
import cv2
from imutils.video import VideoStream
import os
import numpy as np

In [4]:
localPath_labels = "coco.names"
localPath_weights = "yolov3.weights"
localPath_config = "yolov3.cfg"
labels = open(localPath_labels).read().strip().split("\n")
scaling = 0.005
confidence_threshold = 0.5
nms_threshold = 0.005  # Non Maxima Supression Threshold Vlue
model = cv2.dnn.readNetFromDarknet(localPath_config, localPath_weights)

In [10]:
#Start the video streat
cap = VideoStream(src=0).start()
#Getting the layers here
layers_name = model.getLayerNames()

output_layer = [layers_name[i[0] - 1] for i in model.getUnconnectedOutLayers()]
print(model.getLayerNames)


<built-in method getLayerNames of cv2.dnn_Net object at 0x10dc5de90>


In [ ]:
while True:
    frame = cap.read()
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    model.setInput(blob)
    nnoutputs = model.forward(output_layer)
    confidence_scores = []
    box_dimensions = []
    class_ids = []

    for output in nnoutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold :
                box = detection[0:4] * np.array([w, h, w, h])
                (center_x, center_y, width, height) = box.astype("int")
                x = int(center_x - (width / 2))
                y = int(center_y - (height / 2))
                box_dimensions.append([x, y, int(width), int(height)])
                confidence_scores.append(float(confidence))
                class_ids.append(class_id)
    ind = cv2.dnn.NMSBoxes(box_dimensions, confidence_scores, confidence_threshold, nms_threshold)
    for i in ind:
        i = i[0]
        (x, y, w, h) = (box_dimensions[i][0], box_dimensions[i][1],box_dimensions[i][2], box_dimensions[i][3])
        cv2.rectangle(frame,(x, y), (x + w, y + h), (0, 255, 255), 2)
        label = "{}: {:.4f}".format(labels[class_ids[i]], confidence_scores[i])
        cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 2)
    cv2.imshow("Yolo", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()
cap.stop()